# 정적 크롤링 : BeautifulSoup

```
uv add beautifulsoup4
```

* [참고자료, TCP School](https://tcpschool.com/html/html_intro_elementStructure)

# 1. 미니 실습

In [2]:
html = """
<html>
  <body>
    <div id="main" class="content">
      <h1>첫 번째 헤더</h1>
      <h1 class="header">두 번째 헤더</h1>
      <p class="description">설명입니다.</p>
      <a href="https://example1.com" class="link">링크 1</a>
      <a href="https://example2.com" class="link">링크 2</a>
      <a href="https://example3.com" class="external">링크 3</a>
    </div>
    <div id="main2" class="content">
      <div class="items">
        <div>
            <img class="thumbnail" src="/data/picture1.png" />
            <div class="title">타이틀1</div>
            <div class="description">설명1</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture2.png" />
            <div class="title">타이틀2</div>
            <div class="description">설명2</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture3.png" />
            <div class="title test">타이틀3</div>
            <div class="description">설명3</div>
        </div>
      </div>
    </div>
  </body>
</html>
"""

## 1) HTML 파싱 준비

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
print(soup)


<html>
<body>
<div class="content" id="main">
<h1>첫 번째 헤더</h1>
<h1 class="header">두 번째 헤더</h1>
<p class="description">설명입니다.</p>
<a class="link" href="https://example1.com">링크 1</a>
<a class="link" href="https://example2.com">링크 2</a>
<a class="external" href="https://example3.com">링크 3</a>
</div>
<div class="content" id="main2">
<div class="items">
<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
</div>
</div>
</body>
</html>



## 2) 파싱

###
`
soup.find('태그명', class_, id)
`

In [4]:
# 맨 처음에 나오는 class 이름이 title인 div를 추출
soup.find("div", class_="title")

<div class="title">타이틀1</div>

In [9]:
soup.find("div", class_="title").text

'타이틀1'

###
`
soup.find_all('태그명', class_, id)
`

In [10]:
# class 이름이 title인 div를 모두추출
outputs = soup.find_all("div", class_="title")
outputs

[<div class="title">타이틀1</div>,
 <div class="title">타이틀2</div>,
 <div class="title test">타이틀3</div>]

In [11]:
for output in outputs :
    print(output.text)

타이틀1
타이틀2
타이틀3


In [21]:
# "두 번째 헤더" 출력
# 목표 : <h1 class="header">두 번째 헤더</h1>
output = soup.find("h1", class_="header")
print(output.text)

두 번째 헤더


In [24]:
# "설명3" 출력
outputs = soup.find_all("div", class_="description")
print(outputs[-1].text)

설명3


In [27]:
# 태그 a를 모두 출력
outputs = soup.find_all("a")
print(outputs)

[<a class="link" href="https://example1.com">링크 1</a>, <a class="link" href="https://example2.com">링크 2</a>, <a class="external" href="https://example3.com">링크 3</a>]


### 속성 추출

In [32]:
output = outputs[0]
print(output)
print(output.attrs)
print(output["href"])
print(output.attrs["href"])     # "href"라는 키가 없으면 오류
print(output.attrs.get("href")) # "href"라는 키가 없어도 오류가 안남


<a class="link" href="https://example1.com">링크 1</a>
{'href': 'https://example1.com', 'class': ['link']}
https://example1.com
https://example1.com
https://example1.com


In [41]:
# html에서 class 이름이 items 안에 있는 요소들로 데이터 프레임 만든다
# 열 이름은 image, title, description로 설정할 것이다
# image열에는 image의 scr 속성을,
# title열에는 class이름이 title인 요소의 텍스트,
# description열에는 class이름이 description열에는 요소의 텍스트

## 1. 태그가 div이고 class이름이 items인 데이터를 추출
items_html = soup.find("div", class_="items")
# print(items_html)


In [46]:
## 2. items_html에서 div인 것을 추출
### recursive = True
div_outputs = items_html.find_all("div")
for output in div_outputs :
    print(output)
    print("="*50)

<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>


In [51]:
### recursive = False
div_outputs = items_html.find_all("div", recursive=False)
data_list = []
for output in div_outputs :
    temp_dict = {}
    temp_dict["image"] = output.find("img", class_="thumbnail").attrs.get("src")
    temp_dict["title"] = output.find("div", class_="title").text
    temp_dict["description"] = output.find("div", class_="description").text
    data_list.append(temp_dict)
    print(temp_dict)


{'image': '/data/picture1.png', 'title': '타이틀1', 'description': '설명1'}
{'image': '/data/picture2.png', 'title': '타이틀2', 'description': '설명2'}
{'image': '/data/picture3.png', 'title': '타이틀3', 'description': '설명3'}


In [52]:
data_list

[{'image': '/data/picture1.png', 'title': '타이틀1', 'description': '설명1'},
 {'image': '/data/picture2.png', 'title': '타이틀2', 'description': '설명2'},
 {'image': '/data/picture3.png', 'title': '타이틀3', 'description': '설명3'}]

In [53]:
import pandas as pd

data = pd.DataFrame(data_list)
data

,image,title,description
0,/data/picture1.png,타이틀1,설명1
1,/data/picture2.png,타이틀2,설명2
2,/data/picture3.png,타이틀3,설명3


In [54]:
# 미션5: a 태그에서 href를 출력해주세요
## 목표: 맨 처음 나오는 a 태그를 추출 <a href="https://example1.com" class="link">링크 1</a>
print(soup.find("a").attrs)
print(soup.find("a").attrs["href"])
print(soup.find("a").get("href"))
print(soup.find("a")["href"])

{'href': 'https://example1.com', 'class': ['link']}
https://example1.com
https://example1.com
https://example1.com


### `soup.select_one("경로")`

In [56]:
# 링크 1 출력
soup.select_one('div > a')

<a class="link" href="https://example1.com">링크 1</a>

In [63]:
# 링크 2 출력 (div 안에 5번째 자식)
soup.select('div > a:nth-child(5)')

[<a class="link" href="https://example2.com">링크 2</a>]

### `soup.select('경로')`

- 클래스명 : `태그명.클래스이름`
- id명 : `#id이름`

In [55]:
# class가 items인 것 출력
soup.select('div.items')

[<div class="items">
 <div>
 <img class="thumbnail" src="/data/picture1.png"/>
 <div class="title">타이틀1</div>
 <div class="description">설명1</div>
 </div>
 <div>
 <img class="thumbnail" src="/data/picture2.png"/>
 <div class="title">타이틀2</div>
 <div class="description">설명2</div>
 </div>
 <div>
 <img class="thumbnail" src="/data/picture3.png"/>
 <div class="title test">타이틀3</div>
 <div class="description">설명3</div>
 </div>
 </div>]

In [61]:
# <div>
#     <img class="thumbnail" src="/data/picture3.png" />
#     <div class="title test">타이틀3</div>
#     <div class="description">설명3</div>
# </div>

# 이 요소만 출력
soup.select("div.items > div:nth-child(3)")

[<div>
 <img class="thumbnail" src="/data/picture3.png"/>
 <div class="title test">타이틀3</div>
 <div class="description">설명3</div>
 </div>]

# 2. 교보문고 크롤링

## 1) HTML 요청하기

In [ ]:
# https://search.kyobobook.co.kr/search?keyword=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&target=kyobo&gbCode=TOT&page=2

In [ ]:
# import urllib 사용할때
base_url = "https://search.kyobobook.co.kr/search"
keyword = "%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5"
target = "kyobo"
gbCode = "TOT"
page = 1

url = f"{base_url}?keyword={keyword}&target={target}&gbCode={gbCode}&page={page}"

In [68]:
# request 이용

base_url = "https://search.kyobobook.co.kr/search"
keyword = "인공지능"
target = "kyobo"
gbCode = "TOT"
page = 1

params = {
    "keyword" : keyword,
    "target" : target,
    "gbCode" : gbCode,
    "page" : page
}

# User-Agent 입력 (개발자 도구에서 Network 들어간 후 복사해오기)
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

import requests
response = requests.get(base_url, params=params, headers=headers)
response

<Response [200]>

In [67]:
print(response.text)

<!DOCTYPE html>
<html lang="ko"
    data-view="ink"
    data-service="search"
>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    
        
            
            <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1">
        
        
        

    
        <meta name="title" content="인공지능 검색결과 | 교보문고">
        <meta property="og:title" content="인공지능 검색결과 | 교보문고">
    

    
        <meta name="description" content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다.">
        <meta property="og:description" content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다.">
    

    <meta property="og:type" content="website">
    <meta property="og:image" content="https://contents.kyobobook.co.kr/resources/fo/images/common/ink/img_logo_kyobo@2x.png">
    <meta property="og:url" content="http://mobile.kyobobook.co.kr">
    
    



<m

## 2) HTML 파싱 준비

In [69]:
from bs4 import BeautifulSoup

html = response.text
soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html data-service="search" data-view="ink" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" name="viewport"/>
<meta content="인공지능 검색결과 | 교보문고" name="title"/>
<meta content="인공지능 검색결과 | 교보문고" property="og:title"/>
<meta content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다." name="description"/>
<meta content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다." property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="https://contents.kyobobook.co.kr/resources/fo/images/common/ink/img_logo_kyobo@2x.png" property="og:image"/>
<meta content="http://mobile.kyobobook.co.kr" property="og:url"/>
<meta content="date=no" name="format-detection"/>
<meta content="2dlgBOp3K0s6wHjZo_Hkas6yaYPKZIVsmres9vC3F34" name="google-site-verification"/>
<meta conten

## 3) 파싱

In [73]:
# ul 태그 : class가 prod_list
test = soup.find_all("ul", class_="prod_list")
print(len(test))    # len = 1
# 어차피 1개 이므로 find_all 말고 find로 찾아서 리스트에 넣자

prod_list = soup.find("ul", class_="prod_list")
# print(prod_list)

1


In [74]:
# prod_list에서 li태그 : class가 prod_item
prod_item = soup.find_all("li", class_="prod_item")
print(prod_item)

[<li class="prod_item">
<!-- 체크박스 영역 -->
<!-- 체크박스 영역 -->
<span class="form_chk no_label">
<!--교보일때-->
<input class="result_checkbox spec_checkbox" data-bid="9788955685329" data-cdtn-code="008" data-code="KOR" data-comb-code="0" data-grp-code="SGK" data-name="인공지능입문" data-pid="S000217120455" data-prhb-age="0" id="chkSearch_S000217120455" name="001_001_S000217120455_chkSearchNm" type="checkbox"/>
<!--ebook/sam일때-->
<!--핫트랙스일때-->
<!--중고 장터일때-->
<label for="chkSearch_S000217120455"><span class="hidden">상품선택</span></label>
</span>
<!-- // 체크박스 영역 -->
<!-- 상품 영역 -->
<div class="prod_area horizontal">
<div class="prod_thumb_box size_lg">
<a class="prod_link" href="https://product.kyobobook.co.kr/detail/S000217120455" onclick="searchResultItemClickEvent('상품 썸네일', &quot;S000217120455&quot;);">
<span class="img_box">
<img class="prod_img_load" data-kbbfn="s3-image" data-kbbfn-adult="0" data-kbbfn-attr="src" data-kbbfn-bid="9788955685329" data-kbbfn-pid="S000217120455" data-kbbfn-size="200x0" da

In [77]:
one = prod_item[0]
one

<li class="prod_item">
<!-- 체크박스 영역 -->
<!-- 체크박스 영역 -->
<span class="form_chk no_label">
<!--교보일때-->
<input class="result_checkbox spec_checkbox" data-bid="9788955685329" data-cdtn-code="008" data-code="KOR" data-comb-code="0" data-grp-code="SGK" data-name="인공지능입문" data-pid="S000217120455" data-prhb-age="0" id="chkSearch_S000217120455" name="001_001_S000217120455_chkSearchNm" type="checkbox"/>
<!--ebook/sam일때-->
<!--핫트랙스일때-->
<!--중고 장터일때-->
<label for="chkSearch_S000217120455"><span class="hidden">상품선택</span></label>
</span>
<!-- // 체크박스 영역 -->
<!-- 상품 영역 -->
<div class="prod_area horizontal">
<div class="prod_thumb_box size_lg">
<a class="prod_link" href="https://product.kyobobook.co.kr/detail/S000217120455" onclick="searchResultItemClickEvent('상품 썸네일', &quot;S000217120455&quot;);">
<span class="img_box">
<img class="prod_img_load" data-kbbfn="s3-image" data-kbbfn-adult="0" data-kbbfn-attr="src" data-kbbfn-bid="9788955685329" data-kbbfn-pid="S000217120455" data-kbbfn-size="200x0" dat

In [91]:
info_box = one.select("#shopData_list > ul > li:nth-child(1) > div.prod_area.horizontal > div.prod_info_box")
info_box

[<div class="prod_info_box">
 <!-- 라벨 -->
 <!-- 라벨 페이지 -->
 <div class="prod_badge">
 <!--교보일때-->
 <!--종이책, 컬쳐(유료), GIFT일때-->
 <!-- DESC : 썸네일로 보기 타입에서 노출되는 뱃지 [ rep ] class 추가 -->
 <!--ebook/sam 일때-->
 <!--핫트랙스일때-->
 </div>
 <!-- // 라벨 -->
 <!-- 상품명 -->
 <div class="auto_overflow_wrap prod_name_group">
 <div class="auto_overflow_contents">
 <div class="auto_overflow_inner">
 <a class="prod_info" href="https://product.kyobobook.co.kr/detail/S000217120455" onclick="searchResultItemClickEvent('상품 제목', &quot;S000217120455&quot;);">
 <span class="prod_category">[국내도서]</span>
 <span class="prod_label" id="gubun3_S000217120455">예약판매</span>
 <span id="cmdtName_S000217120455">인공지능입문</span>
 </a>
 </div>
 </div>
 <div class="auto_overflow_footer">
 <button class="btn_more_detail" data-role="btn-dialog" data-target="#popAllProdTitle" onclick='goTitlePopup("S000217120455");' type="button"><span class="hidden">더보기</span><span class="ico_plus"></span></button>
 <!-- LayerPopup area -->
 <!-- 타이틀 전체

In [95]:
title = info_box[0].select_one("div.auto_overflow_wrap.prod_name_group > div.auto_overflow_contents").text
title.replace("\n", " ").strip()

'[국내도서] 예약판매 인공지능입문'